In [42]:
#include json and os librareries
import json
import time
import pandas as pd
import os
from subprocess import run

In [43]:
####Globals:
JSON_DIR="ExtractTestsResults/JsonTestsFolders/oslats/oslat-"
TEST_TYPE = "oslat"

############################################################################
#                                                                          #
#                    #################################                     #
#                    #      Helper functions         #                     #
#                    #################################                     #
#                                                                          #
############################################################################

def create_json(results_dict:dict, JSON_DIR:str):
    #create json object
    #add uniuqe id to file name (using unix time)
    timestamp = str(time.time()).replace('.', '')
    # open a file and write the JSON data to it
    with open(JSON_DIR+timestamp+".json", "w") as json_file:
        json.dump(results_dict, json_file)

def list_files(path):
    """
    Return a list of all json file names in the specified folder.
    """
    files = []
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            files.append(file)
    return files

def extract_test_units(results_dict:dict, test_units_df):
    """
    extract all test units metrics(thread/cores) of a specific test, and update them in the result dict.\n
    params:
        -results_dict(dict)-dict that contains test parameters and metrics.\n
        -test_units_df(dataframe)- dataframe that contains specific test's test units metrics.\n
    returns: updated dict that contains test parameters and metrics.\n.
    """
    test_units_list = []
    for _, row in test_units_df.iterrows():
        test_units_list.append({"index":row["Index"], 
            "max_latency":row["Max"], 
            "min_latency":row["Min"], 
            "avg_latency":row["Avg"], 
            "availability":row["Availability"], 
            "number_of_nines":row["Number of Nines"]})

    results_dict["test_units"] = test_units_list

    return results_dict   


In [44]:
availabilty_df = pd.read_csv("spreadsheets/Informal - RAN QE Performance KPIs - oslat.csv").fillna("")
#availabilty_df = availabilty_df.loc[~availabilty_df["OCP Version"].isin(['4.11.0-rc.1', '4.10.23', '4.11.0-rc.5', '4.11.6',
#      '4.12.0-0.nightly-2022-10-24-103753-spr',
#       '4.12.0-0.nightly-2022-10-24-103753',
#       '4.12.0-0.nightly-2022-10-24-103753-telco', '4.12.0-rc.1',
#       '4.13.0-0.nightly-2023-02-16-120330',
#       '4.13.0-0.ci.test-2023-03-02-214629-ci-ln-xw171tb-latest'])]

availabilty_df["Version"] = availabilty_df["Version"].astype("str")
availabilty_df["Operator Version"] = availabilty_df["Operator Version"].astype("str")
availabilty_df.loc[availabilty_df["Version"] == "4.1", "Version"] = "4.10"
availabilty_df.loc[availabilty_df["Operator Version"] == "4.1", "Operator Version"] = "4.10"
if not set(["Kernel", "Sideloaded"]).issubset(availabilty_df.columns):
    availabilty_df["Kernel"] = ""
    availabilty_df["Sideloaded"] = ""

In [45]:
availabilty_df

,Version,OCP Version,Operator Version,Index,Max,Min,Avg,Availability,Number of Nines,Cluster,Duration,Kernel,Sideloaded
0,4.11,4.11.0-rc.5,4.11,3,7,1,1,100.0,100,worker-1,8h,,No
1,4.11,4.11.0-rc.5,4.11,4,8,1,1,100.0,100,worker-1,8h,,No
2,4.11,4.11.0-rc.5,4.11,5,7,1,1,100.0,100,worker-1,8h,,No
3,4.11,4.11.0-rc.5,4.11,6,7,1,1,100.0,100,worker-1,8h,,No
4,4.11,4.11.0-rc.5,4.11,0,8,1,1,100.0,100,worker-1,8h,,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,4.12,4.12.6-SPR,4.12,11,12,1,1,100.0,100,cnfde30,8h,4.18.0-372.46.1.el8_6.x86_64,No
150,4.12,4.12.6-SPR,4.12,12,12,1,1,100.0,100,cnfde30,8h,4.18.0-372.46.1.el8_6.x86_64,No
151,4.12,4.12.6-SPR,4.12,13,11,1,1,100.0,100,cnfde30,8h,4.18.0-372.46.1.el8_6.x86_64,No
152,4.12,4.12.6-SPR,4.12,1,11,1,1,100.0,100,cnfde30,8h,4.18.0-372.46.1.el8_6.x86_64,No


In [46]:
grouped_availabilty = availabilty_df.groupby(["Version", "OCP Version"])

In [47]:
for version_key, threads in grouped_availabilty:
    print("creating json file for {}".format(version_key))
    threads = threads.sort_values(by='Index', ascending=True)
    results_dict = {}
    results_dict["ocp_version"] = threads["Version"].values[0]
    results_dict["ocp_build"] = threads["OCP Version"].values[0]
    results_dict["test_type"] = TEST_TYPE
    results_dict["node_name"] = threads["Cluster"].values[0]
    results_dict["duration"] = threads["Duration"].values[0]
    results_dict["kernel"] = threads["Kernel"].values[0]
    results_dict["sideloaded"] = "true" if threads["Sideloaded"].values[0] == "Yes" else "false" if threads["Sideloaded"].values[0] == "No" else "" 
    results_dict["operator_version"] = threads["Operator Version"].values[0]

    results_dict = extract_test_units(results_dict=results_dict, test_units_df=threads)
    create_json(results_dict=results_dict, JSON_DIR=JSON_DIR)

creating json file for ('4.11', '4.11.0-rc.5')
creating json file for ('4.11', '4.11.6')
creating json file for ('4.12', '4.12.0')
creating json file for ('4.12', '4.12.0-0.nightly-2022-10-24-103753')
creating json file for ('4.12', '4.12.0-0.nightly-2022-10-24-103753-spr')
creating json file for ('4.12', '4.12.0-0.nightly-2022-10-24-103753-telco')
creating json file for ('4.12', '4.12.0-rc.1')
creating json file for ('4.12', '4.12.0-rc.7')
creating json file for ('4.12', '4.12.6-SPR')
creating json file for ('4.13', '4.13.0-0.ci.test-2023-03-02-214629-ci-ln-xw171tb-latest')
creating json file for ('4.13', '4.13.0-0.nightly-2023-02-16-120330')


In [41]:
json_files = list_files("ExtractTestsResults/JsonTestsFolders/oslats")

for file in json_files:
    json_file_file = "ExtractTestsResults/JsonTestsFolders/oslats/" + file
    cmd = ['python3', 'splunkHecPush.py',  '-j', json_file_file, '-i', 'ecosystem-qe-dev', '-t' ,'94eeb2f9-8e5f-4c86-adec-336b3368c74e']
    print("pushing {} to splunk".format(file))
    run(cmd)

pushing cyclictest-16771495664282587.json to splunk


/home/alugasi/tasks/google data studio/code/data/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'splunk-hec.prod.utility-us-east-2.redhat.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


(200, '{"text":"Success","code":0}', {'Date': 'Tue, 14 Mar 2023 09:10:13 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '27', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALB=5ygWzkHvWEjLx+sOSVO82gFANrJBw3w9DBxjq3T2WVEn5afW2Cz+VCZyy/iN87wCvyrbWdv20mme8DlIK2YI/6D9dq6xcLHHi7IXgCwGavnSCinUwHEzEc8tbwjW; Expires=Tue, 21 Mar 2023 09:10:13 GMT; Path=/, AWSALBCORS=5ygWzkHvWEjLx+sOSVO82gFANrJBw3w9DBxjq3T2WVEn5afW2Cz+VCZyy/iN87wCvyrbWdv20mme8DlIK2YI/6D9dq6xcLHHi7IXgCwGavnSCinUwHEzEc8tbwjW; Expires=Tue, 21 Mar 2023 09:10:13 GMT; Path=/; SameSite=None; Secure', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Authorization', 'X-Frame-Options': 'SAMEORIGIN', 'Server': 'Splunkd'})
pushing cyclictest-1678781760290857.json to splunk


/home/alugasi/tasks/google data studio/code/data/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'splunk-hec.prod.utility-us-east-2.redhat.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


(200, '{"text":"Success","code":0}', {'Date': 'Tue, 14 Mar 2023 09:10:14 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '27', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALB=0INK1zXbcftz3jY+InCP5DWkgSTy3MLPyuNRD6ZEQIHVhmO8LoQdVUJt3V/KUz7Sf6mnXAYyJDw4k3w8b4IAeL5QeigVBPUcsb95O6RkVfGx3zuLZiBYOgd7uDRd; Expires=Tue, 21 Mar 2023 09:10:14 GMT; Path=/, AWSALBCORS=0INK1zXbcftz3jY+InCP5DWkgSTy3MLPyuNRD6ZEQIHVhmO8LoQdVUJt3V/KUz7Sf6mnXAYyJDw4k3w8b4IAeL5QeigVBPUcsb95O6RkVfGx3zuLZiBYOgd7uDRd; Expires=Tue, 21 Mar 2023 09:10:14 GMT; Path=/; SameSite=None; Secure', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Authorization', 'X-Frame-Options': 'SAMEORIGIN', 'Server': 'Splunkd'})
pushing cyclictest-1678781760293547.json to splunk


...

(200, '{"text":"Success","code":0}', {'Date': 'Tue, 14 Mar 2023 09:10:31 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '27', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALB=Ami7wchEafr2b1meCRtuFDSklK3GkXTiX2rg9VL9VM6njJXrmj4UZ16wYNYkXEZvXDt24teehOm6XiDyiyIpfG11+9H6l4pEjtpZGHWH6LUPItk6hv2GyJylwpg4; Expires=Tue, 21 Mar 2023 09:10:31 GMT; Path=/, AWSALBCORS=Ami7wchEafr2b1meCRtuFDSklK3GkXTiX2rg9VL9VM6njJXrmj4UZ16wYNYkXEZvXDt24teehOm6XiDyiyIpfG11+9H6l4pEjtpZGHWH6LUPItk6hv2GyJylwpg4; Expires=Tue, 21 Mar 2023 09:10:31 GMT; Path=/; SameSite=None; Secure', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Authorization', 'X-Frame-Options': 'SAMEORIGIN', 'Server': 'Splunkd'})
pushing cyclictest-16787843904580173.json to splunk


/home/alugasi/tasks/google data studio/code/data/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'splunk-hec.prod.utility-us-east-2.redhat.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


(200, '{"text":"Success","code":0}', {'Date': 'Tue, 14 Mar 2023 09:10:32 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '27', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALB=IRjoRaMZgFVoqqFzePzWXvI3cJ2H/jYyWUb7A9+y3UVlTSHuimomD6kkz1lAdQEQH05hIWBU5I4IMv18GXlBtmpWuZ2EPHlcEjJ1Jyk+P5K/PQSGM6DnXAhalQ8x; Expires=Tue, 21 Mar 2023 09:10:32 GMT; Path=/, AWSALBCORS=IRjoRaMZgFVoqqFzePzWXvI3cJ2H/jYyWUb7A9+y3UVlTSHuimomD6kkz1lAdQEQH05hIWBU5I4IMv18GXlBtmpWuZ2EPHlcEjJ1Jyk+P5K/PQSGM6DnXAhalQ8x; Expires=Tue, 21 Mar 2023 09:10:32 GMT; Path=/; SameSite=None; Secure', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Authorization', 'X-Frame-Options': 'SAMEORIGIN', 'Server': 'Splunkd'})
pushing cyclictest-16787843904595797.json to splunk


/home/alugasi/tasks/google data studio/code/data/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'splunk-hec.prod.utility-us-east-2.redhat.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


(200, '{"text":"Success","code":0}', {'Date': 'Tue, 14 Mar 2023 09:10:34 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '27', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALB=x2EaHfFgR4gQCWEtwLVkp6+TxhokpN2KVZ6vAsaHJhuVkGU7zFke8FdHfHiskL4MG17TA/ONm6O1NeZHyBflQx5giWO9TzvmV4sv135jifhsM6WNDiz1uGC17G6+; Expires=Tue, 21 Mar 2023 09:10:34 GMT; Path=/, AWSALBCORS=x2EaHfFgR4gQCWEtwLVkp6+TxhokpN2KVZ6vAsaHJhuVkGU7zFke8FdHfHiskL4MG17TA/ONm6O1NeZHyBflQx5giWO9TzvmV4sv135jifhsM6WNDiz1uGC17G6+; Expires=Tue, 21 Mar 2023 09:10:34 GMT; Path=/; SameSite=None; Secure', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Authorization', 'X-Frame-Options': 'SAMEORIGIN', 'Server': 'Splunkd'})
